In [1]:
import numpy as np
import librosa
import librosa.feature
import librosa.display
import glob
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils.np_utils import to_categorical

import librosa
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
from tqdm import tqdm
import gc
import IPython.display as ipd
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

MFCC_MAX_LEN = 2000
MFCC_NUM = 20
SAMPLING_RATE = 44100

Using TensorFlow backend.


In [2]:
target_labels = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz','metal', 'pop', 'reggae', 'rock']

In [3]:
def construct_df(target_labels):
    arr = []
    for label in target_labels:
        sound_files = glob.glob('genres/'+label+'/*.wav')
#     print('processing %d songs in %s genre...' % (len(sound_files), genre))
        for f in sound_files:
            arr.append([f, label])
    df = pd.DataFrame(arr, columns=['fname','labels'])
    return df      

In [4]:
train_curated = construct_df(target_labels)
train_curated.groupby("labels").size()
# train_curated.head()

labels
blues        100
classical    100
country      100
disco        100
hiphop       100
jazz         100
metal        100
pop          100
reggae       100
rock         100
dtype: int64

In [5]:
# def wav2mfcc(file_path, max_len=11):
def wav2mfcc(wave, max_len=MFCC_MAX_LEN):
#     mfcc = librosa.feature.mfcc(wave, sr=16000)
#     mfcc = librosa.feature.mfcc(wave, n_mfcc=MFCC_NUM, sr=SAMPLING_RATE)
#     y, sr = librosa.load(wave)
    mfcc = librosa.feature.mfcc(wave)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

In [6]:
def get_label_num(labels):
    lbs = labels.split(",")
#     target_lb = "Accelerating_and_revving_and_vroom"
    target_arr = np.zeros(80)
    for lb in lbs:
        if(lb in target_labels):
            i = target_labels.index(lb)
            target_arr[i] = 1
            break
    return target_arr

In [7]:
X = []
y = []

def append_X_Y(labels, wave):
    y.append(get_label_num(labels))
    mfcc = wav2mfcc(wave)
    X.append(mfcc)

for index, row in tqdm(train_curated.iterrows()):
    labels = row["labels"]
    wave, sr = librosa.load('./' + row["fname"], mono=True, sr=44100)
    wave = wave[::3]
    
#     if(len(labels.split(",")) == 1):
    append_X_Y(labels, wave)

0it [00:01, ?it/s]


ParameterError: Audio buffer is not Fortran-contiguous. Use numpy.asfortranarray to ensure Fortran contiguity.

In [ ]:
def one_hot_labels
# convert labels to one-hot encoding
    label_uniq_ids, label_row_ids = np.unique(all_labels, return_inverse=True)
    label_row_ids = label_row_ids.astype(np.int32, copy=False)
    onehot_labels = to_categorical(label_row_ids, len(label_row_ids))

In [ ]:
df = construct_df(target_labels)
df.head()

In [52]:
df["label_count"] = df["labels"].apply(count_labels)
df["target_label_count"] = df["labels"].apply(count_target_labels)
df.head()

,fname,labels,label_count,target_label_count
0,genres/blues/blues.00093.wav,blues,1,1
1,genres/blues/blues.00087.wav,blues,1,1
2,genres/blues/blues.00050.wav,blues,1,1
3,genres/blues/blues.00044.wav,blues,1,1
4,genres/blues/blues.00078.wav,blues,1,1


In [ ]:
def count_labels(labels):
    array_lbs = labels.split(",")
    return len(array_lbs)

def count_target_labels(labels):
    count = 0
    array_lbs = labels.split(",")
    for lb in array_lbs:
        if lb in target_labels:
            count += 1
    return count

In [22]:
def extract_features_song(wave, max_len=MFCC_MAX_LEN):
#     mfcc = librosa.feature.mfcc(wave, sr=16000)
    mfcc = librosa.feature.mfcc(wave, n_mfcc=MFCC_NUM, sr=SAMPLING_RATE)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    # normalize mfcc
    mfcc /= np.amax(np.absolute(mfcc))
    
    return mfcc

In [23]:
def generate_features_and_labels():
    all_features = []
    all_labels = []
    
    genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz','metal', 'pop', 'reggae', 'rock']
    for genre in genres:
        sound_files = glob.glob('genres/'+genre+'/*.wav')
        print('processing %d songs in %s genre...' % (len(sound_files), genre))
        for f in sound_files:
            features = extract_features_song(f)
            all_features.append(features)
            all_labels.append(genre)
            
# convert labels to one-hot encoding
    label_uniq_ids, label_row_ids = np.unique(all_labels, return_inverse=True)
    label_row_ids = label_row_ids.astype(np.int32, copy=False)
    onehot_labels = to_categorical(label_row_ids, len(label_row_ids))
#     print(np.stack(all_features), onehot_labels)
#     print(onehot_labels)
    return np.stack(all_features), onehot_label    

In [24]:
features, labels = generate_features_and_labels()

processing 100 songs in blues genre...


ParameterError: Audio data must be of type numpy.ndarray